# Capstone Project - Segmentation and Clustering Neighborhoods in the city of Toronto, Canada

## Table of contents

- Introduction: Business Problem
- Data
- Answer1: Scraping the wikipedia page
- Answer2: Get the coordinates of each neighborhood
- Answer3: Explore and cluster the neighborhoods in Toronto
- Conclusion

## Introduction: Business Problem
This analysis shows that the busiest area in the city of Toronto is the neighborhoods connected to the only boroughs that contain the word Toronto to analyze the segmentation and clustering neighborhoods.

## Data 
Based on definition of our problem, factors that will influence our decision are:
- The number of existing venues in the neighborhood
- The most common venues in the neighborhood

Following data sources will be needed to extract/generate the required information:
- Information such as the postal code, borough, and neighborhood of the city of Toronto will be collected by scraping the wikipedia page.
- Toronto Neighborhood coordinates will be collected using the prepared geospatial data csv file.
- The latitude and longitude values of the city of Toronto will be collected using the geopy library.
- The number, type, location, postal code and distance of the venues in each neighborhood will be obtained using the Foursquare API.

### Import the necessary libraries required

In [1]:
# Download the geocoding library
!pip install geocoder
!pip install geopy

# Download the folium library to visualize geospatial data
!pip install folium

In [2]:
# For web scrapping.
from bs4 import BeautifulSoup

# Download a web page
import requests

# For data manipulation and analysis
import pandas as pd

# For computing data
import numpy as np

# Import geocoder
import geocoder 

# Map rendering library
import folium 

In [3]:
# Converting an address into latitude and longitude values
from geopy.geocoders import Nominatim  

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# Import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist

from folium import plugins
from folium.plugins import HeatMap

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

### Scraping the Wikipedia page 
This [Wikipedia](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) page contains an html table with data of postal codes of Toronto, Canada. 

Firstly we will get the contents of the webpage in text format and store in a variable called data. Then we will use BeautifulSoup constructor to find the necessary html table in the web page.

In [4]:
# The URL contains an HTML table with data of postal codes of Canada: M.
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [5]:
# Getting the contents of the webpage in text format and store in a variable called data
data  = requests.get(url).text

In [6]:
# Passing the data into the BeautifulSoup constructor
soup = BeautifulSoup(data,"html5lib")

In [7]:
# Find a html table in the web page
table = soup.find('table')

### Creating the Dataset
After finding the html table, we create the dataframe. As seen on the web page, there are 'Not assigned' values in the table. We extract these values while creating the dataframe. Also, we clear the symbols like '/, (,)' in the table. And finally, we correct the names of some boroughs that are incorrectly named. Thus, we obtain the dataframe formed in Postal Code, Borough, Neighborhood columns.

In [8]:
table_contents=[]
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        cell['Postal Code'] = row.p.text[:3]
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto',
                                             'Queen\'s Park':'Downtown Toronto',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto',
                                             'EtobicokeNorthwest':'Etobicoke','East YorkEast Toronto':'East York',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

Checking for duplicate rows based on the Postal Code column

In [9]:
# Select all duplicate rows based on one column
duplicateRowsDF = df[df.duplicated(['Postal Code'])]

print("Duplicate Rows based on a single column are:", duplicateRowsDF, sep='\n')

Duplicate Rows based on a single column are:
Empty DataFrame
Columns: [Borough, Neighborhood, Postal Code]
Index: []


Let's now examine the Dataframe that we just created!!!

In [10]:
df.head(11)

,Borough,Neighborhood,Postal Code
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,"Regent Park, Harbourfront",M5A
3,North York,"Lawrence Manor, Lawrence Heights",M6A
4,Downtown Toronto,Ontario Provincial Government,M7A
5,Etobicoke,Islington Avenue,M9A
6,Scarborough,"Malvern, Rouge",M1B
7,North York,Don Mills North,M3B
8,East York,"Parkview Hill, Woodbine Gardens",M4B
9,Downtown Toronto,"Garden District, Ryerson",M5B


In [11]:
df.shape

(103, 3)

### Get the coordinates of each neighborhood 
Using the given [**Geospatial Coordinates.csv**](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv)  file, we can now get the latitude and longitude values of each neigborhood.

In [12]:
#df_coordinates = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
df_coordinates = pd.read_csv('Geospatial_Coordinates.csv')
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now that we have the coordinates of each neighborhood, we can create the data frame accordingly. We can use the pandas' merge function and then use the **Postal Code** column to match the two tables.

In [13]:
merged_df = pd.merge(df,df_coordinates,on='Postal Code')
merged_df.head(10)

,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,North York,Parkwoods,M3A,43.753259,-79.329656
1,North York,Victoria Village,M4A,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,Downtown Toronto,Ontario Provincial Government,M7A,43.662301,-79.389494
5,Etobicoke,Islington Avenue,M9A,43.667856,-79.532242
6,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
7,North York,Don Mills North,M3B,43.745906,-79.352188
8,East York,"Parkview Hill, Woodbine Gardens",M4B,43.706397,-79.309937
9,Downtown Toronto,"Garden District, Ryerson",M5B,43.657162,-79.378937


We remove the neighborhoods that do not belong to the City of Toronto from the dataframe.

In [14]:
index = merged_df[merged_df['Borough'] == 'Mississauga'].index
merged_df.drop(index,axis=0,inplace=True)

Now, let's examine the merged dataframe.

In [15]:
print(
    'The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(merged_df['Borough'].unique()),
        len(merged_df['Neighborhood'].unique())
    )
)

The dataframe has 9 boroughs and 102 neighborhoods.


In [16]:
merged_df.head()

,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,North York,Parkwoods,M3A,43.753259,-79.329656
1,North York,Victoria Village,M4A,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,Downtown Toronto,Ontario Provincial Government,M7A,43.662301,-79.389494


In [17]:
merged_df_group= merged_df.groupby(['Borough'])
merged_df_group.mean()

,Latitude,Longitude
Borough,,
Central Toronto,43.701980,-79.398954
Downtown Toronto,43.654597,-79.383972
East Toronto,43.669436,-79.324654
East York,43.700303,-79.335851
Etobicoke,43.660043,-79.542074
North York,43.750727,-79.429338
Scarborough,43.766229,-79.249085
West Toronto,43.652653,-79.449290
York,43.690797,-79.472633


### Explore and Cluster the Toronto Neighborhoods

So, let's visualize the data to better understand them by using the geopy library to get Toronto's latitude and longitude values.

In [18]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [19]:
# create map of Toronto using latitude and longitude values
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Borough'], merged_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='navy',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

### Get only boroughs that contain the word Toronto

In [20]:
Toronto_df = merged_df[merged_df['Borough'].str.contains('Toronto')].reset_index(drop=True)
Toronto_df.head(10)

,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
1,Downtown Toronto,Ontario Provincial Government,M7A,43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",M5B,43.657162,-79.378937
3,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418
4,East Toronto,The Beaches,M4E,43.676357,-79.293031
5,Downtown Toronto,Berczy Park,M5E,43.644771,-79.373306
6,Downtown Toronto,Central Bay Street,M5G,43.657952,-79.387383
7,Downtown Toronto,Christie,M6G,43.669542,-79.422564
8,Downtown Toronto,"Richmond, Adelaide, King",M5H,43.650571,-79.384568
9,West Toronto,"Dufferin, Dovercourt Village",M6H,43.669005,-79.442259


In [21]:
# create map of boroughs that contain the word Toronto using latitude and longitude values
Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto)  
    
Toronto

### Foursquare
Now that we have coordinates of the neighborhoods in Toronto, let's use Foursquare API to get info on various kinds of venues in each neighborhood.

Then, we extract the name, latitude and longitude, postal code and distance of the venues from the json file we received using the foursquare api and collect it in a dataframe.

### Explore Neighborhoods in the Manhatonly boroughs that contain the word 'Torontotan'

In [22]:
CLIENT_ID = 'JEJBVSCCNROEXDAAECHR0TQ4ZFKTM13LRXGDNQCJZAZVW0IZ' # your Foursquare ID
CLIENT_SECRET = 'MMG3PVWMJFZOTXHGOOQQ2GKP2Y0QRTZOSQH4BDC3GU2V4O2U' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 1000 # A default Foursquare API limit value
CATEGORY = "4d4b7105d754a06374d81259"
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JEJBVSCCNROEXDAAECHR0TQ4ZFKTM13LRXGDNQCJZAZVW0IZ
CLIENT_SECRET:MMG3PVWMJFZOTXHGOOQQ2GKP2Y0QRTZOSQH4BDC3GU2V4O2U


In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
# get all the venues
all_venues = getNearbyVenues(
                             names=Toronto_df['Neighborhood'],
                             latitudes=Toronto_df['Latitude'],
                             longitudes=Toronto_df['Longitude']
                            )

Regent Park, Harbourfront
Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
F

Let's examine the venues dataframe.

In [25]:
print('Shape of the dataframe : {}'.format(all_venues.shape))

Shape of the dataframe : (1614, 7)


In [26]:
all_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
5,"Regent Park, Harbourfront",43.65426,-79.360636,Corktown Common,43.655618,-79.356211,Park
6,"Regent Park, Harbourfront",43.65426,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub
7,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
8,"Regent Park, Harbourfront",43.65426,-79.360636,The Extension Room,43.653313,-79.359725,Gym / Fitness Center
9,"Regent Park, Harbourfront",43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site


In [27]:
all_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",18,18,18,18,18,18
Central Bay Street,63,63,63,63,63,63
Christie,16,16,16,16,16,16
Church and Wellesley,79,79,79,79,79,79
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35
Davisville North,7,7,7,7,7,7


In [28]:
print('There are {} uniques categories!!!'.format(len(all_venues['Venue Category'].unique())))

There are 238 uniques categories!!!


### Analysis 
As seen in the table, the venue postal code is given in more detail than the neighborhood postal code. Since we will make matches using postal codes, we first make them the same type.

#### Let's explore the first neighborhood in our dataframe

In [29]:
Toronto_df.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

#### **Get the neighborhood's latitude and longitude values**

In [30]:
neighborhood_latitude = Toronto_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Toronto_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Toronto_df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


#### Now, let's get the top 50 venues that are in Regent Park, Harbourfront within the radius of 500 meters

In [31]:
LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=JEJBVSCCNROEXDAAECHR0TQ4ZFKTM13LRXGDNQCJZAZVW0IZ&client_secret=MMG3PVWMJFZOTXHGOOQQ2GKP2Y0QRTZOSQH4BDC3GU2V4O2U&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=50'

In [32]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60f2cdbe85ade9406e3f0b6a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 46,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

In [33]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [34]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


In [35]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

46 venues were returned by Foursquare.


## Analyze Each Neighborhood

In [36]:
# one hot encoding
toronto_onehot = pd.get_dummies(all_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to the dataframe
toronto_onehot['Neighborhood'] = all_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
toronto_onehot.shape

(1614, 238)

Next, group the rows by neighborhood and the mean of the frequency of occurrence of each category

In [38]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.041667,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,Central Bay Street,0.015873,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.015873,0.0,0.0,0.0
4,Christie,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [39]:
toronto_grouped.shape

(38, 238)

## Displaying each neighborhood along with the top 5 most common venues

In [40]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0  Cocktail Bar  0.05
1        Bakery  0.05
2   Coffee Shop  0.05
3      Beer Bar  0.03
4    Restaurant  0.03


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0                    Café  0.12
1             Coffee Shop  0.08
2          Breakfast Spot  0.08
3              Restaurant  0.04
4  Furniture / Home Store  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.17
1    Airport Lounge  0.11
2  Airport Terminal  0.11
3          Boutique  0.06
4   Harbor / Marina  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1      Sandwich Place  0.05
2                Café  0.05
3  Italian Restaurant  0.05
4        Burger Joint  0.03


----Christie----
                venue  freq
0       Grocery Store  0.25
1                Café  0.19
2                Park  0.12


### Adding all that into a pandas dataframe

#### First, let's write a function to sort the venues in descending order.

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Bakery,Cocktail Bar,Coffee Shop,Beer Bar,Pub,Farmers Market,Seafood Restaurant,Restaurant,Pharmacy,Cheese Shop
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Performing Arts Venue,Bar,Restaurant,Burrito Place,Climbing Gym,Pet Store,Nightclub
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Bar,Plane,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry
3,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Salad Place,Japanese Restaurant,Bubble Tea Shop,Burger Joint,Miscellaneous Shop,Juice Bar
4,Christie,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports,Nightclub,Coffee Shop


### Cluster Neighborhoods

First, we cluster neighborhoods together to see similar neighborhoods. This will enables us to offer our stakeholders different neighborhoods as an alternative. We start by preparing our dataframe.

In [43]:
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
toronto_grouped_clustering.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.0
1,0.041667,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.000000,0.0,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.015873,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.015873,0.0,0.0,0.0
4,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [44]:
# set number of clusters
kclusters = 4

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0, n_init=50).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Checking the number of neighborhoods

In [45]:
print(
    'The dataframe has {} neighborhoods.'.format(
        len(toronto_grouped['Neighborhood'].unique())
    )
)

The dataframe has 38 neighborhoods.


As we have a lost neighborhood, we need to make the dataframe more useful. Therefore, we need to remove the missing neighborhood.

In [46]:
neigh = []
index_list = Toronto_df['Neighborhood'].tolist()
for ind in index_list:
    b = toronto_grouped[toronto_grouped['Neighborhood'] == ind ]
    if b.empty:
        neigh.append(ind)

print(neigh)

['Moore Park, Summerhill East']


In [47]:
merged_data = Toronto_df.drop('Postal Code', 1)
index = merged_data[merged_data['Neighborhood'] == 'Moore Park, Summerhill East'].index
merged_data.drop(index,axis=0,inplace=True)
merged_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,Downtown Toronto,Ontario Provincial Government,43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,Downtown Toronto,St. James Town,43.651494,-79.375418
4,East Toronto,The Beaches,43.676357,-79.293031


Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [48]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = Toronto_df
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head()

,Borough,Neighborhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636,0.0,Coffee Shop,Pub,Bakery,Park,Café,Breakfast Spot,Theater,Gym / Fitness Center,Event Space,Playground
1,Downtown Toronto,Ontario Provincial Government,M7A,43.662301,-79.389494,0.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Burrito Place,Spa,Smoothie Shop,Beer Bar,Italian Restaurant,Sandwich Place,Distribution Center
2,Downtown Toronto,"Garden District, Ryerson",M5B,43.657162,-79.378937,0.0,Coffee Shop,Clothing Store,Japanese Restaurant,Cosmetics Shop,Café,Bubble Tea Shop,Middle Eastern Restaurant,Lingerie Store,Pizza Place,Italian Restaurant
3,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418,0.0,Coffee Shop,Café,Cosmetics Shop,Restaurant,Cocktail Bar,Bakery,Seafood Restaurant,Park,Moroccan Restaurant,Clothing Store
4,East Toronto,The Beaches,M4E,43.676357,-79.293031,0.0,Pub,Trail,Health Food Store,Deli / Bodega,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


Finally, we can visualize the resulting clusters!!!

In [49]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [50]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow,
        fill=True,
        fill_color=rainbow,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

We've finally finished clustering the neighborhoods. So we found out which neighborhoods were similar. In this way, we offered different alternatives to stakeholders.

## Examine Clusters
Now, we examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can assign a name to each cluster.

### Cluster 1 => Downtown Area

In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",0.0,Coffee Shop,Pub,Bakery,Park,Café,Breakfast Spot,Theater,Gym / Fitness Center,Event Space,Playground
1,Ontario Provincial Government,0.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Burrito Place,Spa,Smoothie Shop,Beer Bar,Italian Restaurant,Sandwich Place,Distribution Center
2,"Garden District, Ryerson",0.0,Coffee Shop,Clothing Store,Japanese Restaurant,Cosmetics Shop,Café,Bubble Tea Shop,Middle Eastern Restaurant,Lingerie Store,Pizza Place,Italian Restaurant
3,St. James Town,0.0,Coffee Shop,Café,Cosmetics Shop,Restaurant,Cocktail Bar,Bakery,Seafood Restaurant,Park,Moroccan Restaurant,Clothing Store
4,The Beaches,0.0,Pub,Trail,Health Food Store,Deli / Bodega,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
5,Berczy Park,0.0,Bakery,Cocktail Bar,Coffee Shop,Beer Bar,Pub,Farmers Market,Seafood Restaurant,Restaurant,Pharmacy,Cheese Shop
6,Central Bay Street,0.0,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Salad Place,Japanese Restaurant,Bubble Tea Shop,Burger Joint,Miscellaneous Shop,Juice Bar
7,Christie,0.0,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports,Nightclub,Coffee Shop
8,"Richmond, Adelaide, King",0.0,Coffee Shop,Café,Restaurant,Thai Restaurant,Clothing Store,Deli / Bodega,Gym,Bakery,Bar,Concert Hall
9,"Dufferin, Dovercourt Village",0.0,Bakery,Pharmacy,Park,Pizza Place,Brewery,Bar,Bank,Café,Supermarket,Playground


### Cluster 2 => Quiet Rural Area

In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lawrence Park,1.0,Park,Swim School,Bus Line,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


### Cluster 3 => Retirement Residential Area

In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,2.0,Home Service,Garden,Wine Bar,Department Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


### Cluster 4 => Pedestrian Area

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Forest Hill North & West,3.0,Sushi Restaurant,Park,Trail,Jewelry Store,Wine Bar,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
33,Rosedale,3.0,Park,Trail,Playground,Deli / Bodega,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


## Conclusion 
In conclusion, my analysis showed that the busiest areas in the city of Toronto are the neighborhoods connected to the only boroughs that contain the word **"Toronto"**. I got the various kinds of venues in each neighborhood for these area.

The aim of this project is to segment and cluster the neighborhoods in the city of Toronto Canada. Among the total 9 boroughs, I analyze only the boroughs that contains the word ‘**Toronto**’. Therefore, there are only 4 boroughs which include the word ‘Toronto’ which are downtown, East, West and Central Toronto.

According to the K-means clustering analysis conducted, I defined the 4 areas of neighborhood for these areas as the following. I examined each cluster and determined the discriminating venue categories that distinguish each cluster. Based on the defining categories, I then assigned a name to each cluster. 

#### **These are the following neighborhoods area:**

(1) **Downtown Area** where the urban major metro area with lots of restaurants.

(2) **Quiet Rural Area** where its miles from the downtown area and will be perfect for a new family with small children.

(3) **Retirement Residential Area** as its near a home service and garden for the boomers who need a some peace and quiet. 

(4) **Pedestrian Area** is perfect for single professionals as this area is near the downtown with very little noise and density.